In [16]:
import numpy as np
import csv
import math

# Part 1


|Size |Orbit|habitable | Count|
|-----|-----|----------|------|
|big|near|yes|20|
|big|far|yes|170|
|small|near|yes|139|
|small|far|yes|45|
|big|near|no|130|
|big|far|no|30|
|small|near|no|11|
|small|far|no|255|

In [23]:
tree_data = np.array([[1,0,1,20],
                      [1,1,1,170],
                      [0,0,1,139],
                      [0,1,1,45],
                      [1,0,0,130],
                      [1,1,0,30],
                      [0,0,0,11],
                      [0,1,0,255]
                     ])
def calculate_entropy(col_num,data):
    pos_instances = 0
    neg_instances = 0
    for row in data:
        #is habitible
        if(row[-2]==1):
            pos_instances+=row[3]
        else:
            neg_instances+=row[3]
    total = pos_instances+neg_instances
    p_p = float(pos_instances)/float(total)
    p_n = float(neg_instances)/float(total)
    return -1.0*p_p*math.log(p_p,2)-1.0*p_n*math.log(p_n,2)
print(calculate_entropy(0,tree_data))
print(calculate_entropy(1,tree_data))
print(calculate_entropy(2,tree_data))
        

0.9886994082884974
0.9544340029249649
0.9969501570069956


# Part 2

In [1]:

labels=[]
with open("wine_train_set.csv") as csv_file:
    train_data = csv.reader(csv_file, delimiter=',', quotechar='|')
    labels = train_data.__next__()
print(labels)
my_data = np.loadtxt('wine_train_set.csv', delimiter=',',skiprows=1)
print(my_data)
print(my_data.shape[1])
MAX_QUALITY=10
MIN_QUALITY=0

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
[[ 5.6   0.21  0.24 ...  0.31 11.5   7.  ]
 [ 6.2   0.3   0.31 ...  0.39 12.9   7.  ]
 [ 6.4   0.27  0.32 ...  0.48  9.4   5.  ]
 ...
 [ 8.9   0.32  0.49 ...  0.34  9.4   5.  ]
 [ 6.4   0.25  0.32 ...  0.58 10.8   7.  ]
 [ 7.7   0.38  0.23 ...  0.41 13.6   6.  ]]
12


In [2]:
Y_Number=[]
for i in range(0,11):
    Y_Number.append(0)

In [3]:
for row in my_data:
    quality = row[11]
    Y_Number[(int(quality))]+=1
print(Y_Number)

[0, 0, 0, 16, 130, 1166, 1758, 704, 140, 4, 0]


In [4]:
"""Takes array type and returns variance"""
def calculate_variance(data):
    sum = 0.0
    for i in data:
        sum+=float(i**2.0)
    return sum/float(len(data))
"""Takes array type and returns mean"""
def calculate_mean(data):
    sum = 0.0
    for i in data:
        sum+=float(i)
    return sum/float(len(data))

# Calculating Y Priors

In [5]:
Y_Prior=[]
num_samples = my_data.shape[0]
for i in range(0,11):
    Y_Prior.append(0)
for i in range(0,11):
    Y_Prior[i]=Y_Number[i]/num_samples
    print("Y prior "+str(labels[i])+": "+ str(Y_Prior[i]))

Y prior fixed acidity: 0.0
Y prior volatile acidity: 0.0
Y prior citric acid: 0.0
Y prior residual sugar: 0.00408371618172537
Y prior chlorides: 0.033180193976518634
Y prior free sulfur dioxide: 0.29760081674323635
Y prior total sulfur dioxide: 0.44869831546707506
Y prior density: 0.17968351199591628
Y prior pH: 0.03573251659009699
Y prior sulphates: 0.0010209290454313426
Y prior alcohol: 0.0


In [6]:
#printing variances
for i in range(0,10):
    print("standard deviation "+labels[i]+"\t"+str(calculate_variance(my_data[:,i])))

standard deviation fixed acidity	47.79219691168966
standard deviation volatile acidity	0.08694115620214321
standard deviation citric acid	0.12626993363961042
standard deviation residual sugar	67.29836013272077
standard deviation chlorides	0.0025488284839203477
standard deviation free sulfur dioxide	1550.031967840735
standard deviation total sulfur dioxide	20951.93440530883
standard deviation density	0.9881509056302004
standard deviation pH	10.192047907095546
standard deviation sulphates	0.2521797856048954


# Filter data to make calculating mean+standard deviation easy

In [7]:
# round data format
# filtered_data[Quality][xi][entry]
def filter_data_by_quality(data):
    out_list = []
    for i in range(MIN_QUALITY,MAX_QUALITY+1):
        out_list.append(np.array([]))
    print(data)
    #for i in range(MIN_QUALITY,MAX_QUALITY+1):
    #    out_list.append(np.array([]))
    
    for quality in range(MIN_QUALITY,MAX_QUALITY+1):
        for row in data:
            #print(row)       
            if(row[-1]==quality):
                try:
                    #print(row[:-1])
                    out_list[quality] = np.vstack((out_list[quality],row[:-1]))
                except:
                    out_list[quality] = row[:-1]
                #out_list[quality].append(row[:-1])
                
    return out_list
#print(filter_data_by_quality(my_data))

# Calculating posteriors

In [8]:

"""Runs fn on result of filtered data"""
def run_on_data(data,fn):
    out = []
    for quality in data:
        to_append = np.array([])
        #transposing data to get column
        for col in quality.T:
            #print("")
            #print(col)
            to_append=np.hstack((to_append,fn(col)))
        out.append(to_append)
    return out

In [9]:
filtered_data = filter_data_by_quality(my_data)
variance = run_on_data(filtered_data,calculate_variance)
mean = run_on_data(filtered_data,calculate_mean)
print("VARIANCE")
print(variance)
print("MEAN")
print(mean)

[[ 5.6   0.21  0.24 ...  0.31 11.5   7.  ]
 [ 6.2   0.3   0.31 ...  0.39 12.9   7.  ]
 [ 6.4   0.27  0.32 ...  0.48  9.4   5.  ]
 ...
 [ 8.9   0.32  0.49 ...  0.34  9.4   5.  ]
 [ 6.4   0.25  0.32 ...  0.58 10.8   7.  ]
 [ 7.7   0.38  0.23 ...  0.41 13.6   6.  ]]
VARIANCE
[array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([6.23950000e+01, 1.23557812e-01, 1.13300000e-01, 7.17595312e+01,
       2.25056250e-03, 8.22109375e+03, 4.09319062e+04, 9.89831412e-01,
       1.01937938e+01, 2.20962500e-01, 1.08869375e+02]), array([5.25970000e+01, 1.77401538e-01, 1.16553846e-01, 3.62285000e+01,
       3.40993846e-03, 9.67536538e+02, 1.86287481e+04, 9.88629360e-01,
       1.02195908e+01, 2.41427692e-01, 1.04347154e+02]), array([4.88159005e+01, 1.01710206e-01, 1.33723928e-01, 8.24239537e+01,
       3.35475386e-03, 1.64872556e+03, 2.44984927e+04, 9.90538383e-01,
       1.00486298e+01, 2.40316295e-01, 9.69449296e+01]), array([4.74655830e+01, 7.42747582e-02, 1.28583220e-

In [10]:
# calculating probabilities
for i in range(0,mean.shape[0]):
    for j in range(0,mean.shape[1]):
        prob[i,j]=(1/(math.sqrt(2*math.pi*variance[i,j])))*math.exp(-0.5*())

AttributeError: 'list' object has no attribute 'shape'